In [31]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [32]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [33]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [34]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [35]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [36]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.009s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [41]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.264s

OK


In [42]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [43]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


In [44]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [46]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.219s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [57]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
# Create change address "change_address"

SATS_PER_BTC = 100000000    # A hundred million sats per BTC

passphrase = b'rob.baltzer01@gmail.com ChangeGTvX4Yqy'
secret = little_endian_to_int(hash256(passphrase))
change_priv_key = PrivateKey(secret)
change_address = change_priv_key.point.address(compressed=True, testnet=True)
print (changeAddress)

prev_tx = bytes.fromhex('0f10972fda195c57f2d306632cc849d39d037eec48806f01c34312de63a4eb9b')
prev_index = 0
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_amount = 0.12
change_amount = 0.07
passphrase = b'rob.baltzer01@gmail.com 8ueiRwGTvX4Yqy'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
tx_ins = []
tx_ins.append(TxIn(prev_tx, prev_index))
tx_outs = []
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
target_satoshis = int(target_amount*SATS_PER_BTC)
tx_outs.append(TxOut(target_satoshis, script_pubkey))
h160 = decode_base58(change_address)
script_pubkey = p2pkh_script(h160)
change_satoshis = int(change_amount*SATS_PER_BTC)
tx_outs.append(TxOut(change_satoshis, script_pubkey))
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(tx_obj.sign_input(0, priv))
print(tx_obj.serialize().hex())

# confirmed in txid "51286ab8717d2709182343441e7488582a6bbe1a61b9772603bc9e52d40b798d"

miknzETDYq1SbncjqAV4KuGLsEgz7TVnvR
True
01000000019beba463de1243c3016f8048ec7e039dd349c82c6306d3f2575c19da2f97100f000000006b483045022100a0abe9323f4dafed0273fcaf3808350c8b63612a7b8cc60b1ecb37f8e600a87e022062ba4e42c4bb37fd0b798edd3768aec395565be6f608ea8453151f9e143261ef01210361c8cfed33ef8f63273e30f88c46f4952a5987920fe30432b435498fba4b5342ffffffff02001bb700000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788acc0cf6a00000000001976a91423866e770f6430c9722bb297ccfb592250c04a8988ac00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [79]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Split my change address into two outputs (I couldn't get any faucet coins, so had to improvise at the last minute
passphrase1 = b'rob.baltzer01@gmail.com output1GTvX4Yqy'
secret1 = little_endian_to_int(hash256(passphrase1))
output1_priv_key = PrivateKey(secret1)
output1_address = output1_priv_key.point.address(compressed=True, testnet=True)
print (output1_address)

passphrase2 = b'rob.baltzer01@gmail.com output2a8a0vd0'
secret2 = little_endian_to_int(hash256(passphrase2))
output2_priv_key = PrivateKey(secret2)
output2_address = output2_priv_key.point.address(compressed=True, testnet=True)
print (output2_address)

SATS_PER_BTC = 100000000    # A hundred million sats per BTC

prev_tx = bytes.fromhex('51286ab8717d2709182343441e7488582a6bbe1a61b9772603bc9e52d40b798d')
prev_index = 1
target_address = output1_address
target_amount = 0.035
change_amount = 0.034
passphrase = b'rob.baltzer01@gmail.com ChangeGTvX4Yqy' #oops, address reuse. Already broadcast so run with it
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
tx_ins = []
tx_ins.append(TxIn(prev_tx, prev_index))
tx_outs = []
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
target_satoshis = int(target_amount*SATS_PER_BTC)
tx_outs.append(TxOut(target_satoshis, script_pubkey))
h160 = decode_base58(change_address)
script_pubkey = p2pkh_script(h160)
change_satoshis = int(change_amount*SATS_PER_BTC)
tx_outs.append(TxOut(change_satoshis, script_pubkey))
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(tx_obj.sign_input(0, priv))
print(tx_obj.serialize().hex())

# now coins are split, combine those inputs into my single output

passphrase3 = b'rob.baltzer01@gmail.com my_outputa8a0vd0'
secret3 = little_endian_to_int(hash256(passphrase3))
output3_priv_key = PrivateKey(secret3)
output3_address = output3_priv_key.point.address(compressed=True, testnet=True)
print (output3_address)

prev_tx_1 = bytes.fromhex('1c7b72eda3f1781fd019dec942de5165e06f68c0747237e8b4f97375a4a2a5fe')
prev_index_1 = 0
prev_tx_2 = prev_tx_1
prev_index_2 = 1
target_address = 'n1CqJyqJV7JZpQpM3PLBUq9cTHAhoJmppc'
target_amount = 0.067
secret4 = little_endian_to_int(hash256(passphrase1))
priv4 = PrivateKey(secret=secret4)
secret5 = little_endian_to_int(hash256(b'rob.baltzer01@gmail.com ChangeGTvX4Yqy'))
priv5 = PrivateKey(secret=secret5)
tx_ins = []
tx_ins.append(TxIn(prev_tx_1, prev_index_1))
tx_ins.append(TxIn(prev_tx_2, prev_index_2))
tx_outs = []
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
target_satoshis = int(target_amount*SATS_PER_BTC)
tx_outs.append(TxOut(target_satoshis, script_pubkey))
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(tx_obj.sign_input(0, priv4))
print(tx_obj.sign_input(1, priv5))
print(tx_obj.serialize().hex())

mkb5yb6DTNSct1NDiqEFD25Sw7oKZRifVw
n4EtWFcPjdwZhMANC3VjRYVER7LeGnKwzY
True
01000000018d790bd4529ebc032677b9611abe6b2a5888741e4443231809277d71b86a2851010000006a4730440220362375e1b18486823a8505c5e3b82b9e9b503d142242e60d4f53f38dbf1db8670220039d34e0f2fbf79653f15c5e550abf60849d97136fe6ee31885a579f18301b66012102fb8eca59f9320b7b7570cad250af501916c825f701d310ce998092281e23e148ffffffff02e0673500000000001976a91437a0b47f56a484eb753e05b9cf6125a8c6e4389488ac40e13300000000001976a91423866e770f6430c9722bb297ccfb592250c04a8988ac00000000
n1CqJyqJV7JZpQpM3PLBUq9cTHAhoJmppc
True
True
0100000002fea5a2a47573f9b4e8377274c0686fe06551de42c9de19d01f78f1a3ed727b1c000000006b4830450221009f8a5f956a8e6561c0c6fdde554f9dd792f05e820fe7f605024969a54a328f560220024b753a4cbf2ffd003913e18c4119ef52b69e96373077be85f22763a03ab67e012103263a1c5af294aa6899016bf8f493cb66ba2390c86fc2f75c534d07b3865d371cfffffffffea5a2a47573f9b4e8377274c0686fe06551de42c9de19d01f78f1a3ed727b1c010000006a473044022053bfffddd574defbf20468dab3e957e1e1cd526